In [2]:
!pip install duckdb
!pip install notebook
!pip install jupysql pandas matplotlib duckdb-engine


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
input_path = "./data/geojson/osm21_sports.geojson"
input_filename = os.path.basename(input_path)

In [4]:
import duckdb
from sqlalchemy.engine import create_engine

%load_ext sql
conn = create_engine('duckdb:///data/local.duckdb')
%sql conn --alias duckdb
%config SqlMagic.named_parameters="enabled" 

In [5]:
%%sql
INSTALL psql FROM community;
INSTALL spatial;
LOAD spatial;
LOAD psql;


Running query in 'duckdb'

Success


In [6]:
%%sql
CREATE OR REPLACE TABLE point_data AS
SELECT *
FROM ST_Read("{{input_path}}")

Running query in 'duckdb'

Count


In [7]:

import random
import string

try:
    conn.raw_connection().remove_function('random_string')
except:
    pass

# Define the Python function to generate a random string of length 'n'
def random_string(n: int) -> str:
    return ''.join(random.choices(string.ascii_letters + string.digits, k=n))

# Register the function with DuckDB

conn.raw_connection().create_function('random_string', random_string, side_effects=True)

In [8]:
%sql SELECT random_string(10) AS random_string;

Running query in 'duckdb'

random_string
eL3Qe3bAdO


In [54]:

%%sql
CREATE OR REPLACE TABLE point_data_centroid AS
WITH centroid AS (
    FROM point_data
    SELECT
        rowid,
        centroid: geom.ST_Centroid(),
    ORDER BY hash(rowid + 43) DESC
)
FROM centroid
SELECT
    id: ROW_NUMBER() OVER (ORDER BY centroid),
    location: 'point("' || centroid.ST_X() || ',' || centroid.ST_Y() || '")',
    description: random_string(24)
ORDER BY hash(rowid + 43)

Running query in 'duckdb'

Count


In [55]:
%sql SELECT * FROM point_data_centroid

Running query in 'duckdb'

id,location,description
473471,"point(""-95.46691022823593,30.135154307321766"")",y8PTYeXLZ8K7oBXYgI0SwNhH
1142164,"point(""25.222890448148586,42.48537849597041"")",FCLaDrjjx0hombStNz495XvC
937280,"point(""-87.77203167323923,41.99342675335603"")",LGlTNd6qAiMWixwIXNN3feaq
311383,"point(""19.43227684804893,-33.646660475078896"")",O02RejCidcZg8yshrSWW4rrj
1830552,"point(""16.775558021487925,52.39047440708777"")",TUYrWrLyzTGtEnHZIb7OLTA0
777820,"point(""6.793204023007181,50.69685630334198"")",0K7cvzCiALeDuDpugh7FUWLe
840602,"point(""12.118249200000001,44.8272925"")",23OAVvY70pIeav2JzfNW5Of8
312914,"point(""23.696428362136572,38.03527981363856"")",JejHgPR7t8iKA405ez2tUXni
520921,"point(""-48.591141549999996,-22.36732205"")",RTYtgj8vRVmXfFRBP50yv5nb
788565,"point(""174.71119224999998,-36.79012595"")",ATnmeh04phIFEpPpB08t93Ij


In [56]:
%%sql
SELECT COUNT(*) FROM point_data_centroid as data;

Running query in 'duckdb'

count_star()
1982028


In [64]:
%%sql
SUMMARIZE point_data_centroid

Running query in 'duckdb'

column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
id,BIGINT,1,1982028,1831607,991014.5,572162.3440082379,495675,990850,1486544,1982028,0.00
location,VARCHAR,"point(""-0.0001048,51.4978894"")","point(""99.99990515538256,13.414230211107782"")",2436502,None,None,None,None,None,1982028,0.00
description,VARCHAR,0000kSUmesPGfVi4D2TNloSN,zzzwSKxuwAxSoRNRa4lLcotQ,2000429,None,None,None,None,None,1982028,0.00


In [65]:
export_filename = input_filename.replace(".geojson", ".json")
export_path = "./data/jsonl/" + export_filename

os.makedirs(os.path.dirname(export_path), exist_ok=True)

print(f"Exporting to {export_filename}")
print(f"Exporting to {export_path}")

%sql COPY (SELECT * FROM point_data_centroid) TO '{{export_path}}' (FORMAT JSON);

Exporting to osm21_sports.json
Exporting to ./data/jsonl/osm21_sports.json


Running query in 'duckdb'

Count


In [86]:
# replace "point(\"<valueX>,<valueY>\")" with point("139.6499560172321,35.57251104983658")
new_export_path = export_path.replace(".json", ".adm")
with open(new_export_path, "w") as f:
    for row in open(export_path, "r").readlines():
        new_row = row.replace('location":"point(\\"', 'location":point("').replace('\\")","description"', '"),"description"')
        f.write(new_row)